# FDA NDC ETL Pipeline
- API website: https://open.fda.gov/data/downloads/
- Objective: Collect drug information from the FDA NDC (National Drug Code Directory) API

In [1]:
# Requesting the API and getting a status code
import requests
response = requests.get("https://api.fda.gov/drug/ndc.json")
print(response.status_code)

200


In [2]:
import json
import time
from dotenv import load_dotenv
import os

# load credentials
load_dotenv()
api_key = os.getenv('api_key')

limit = 1000 # I got this number by testing on Postman
all_results = []

# get the total number of results from the request
if response.status_code == 200:
    data = response.json()
    total_results = data['meta']['results']['total']

    # print out the count
    print(f"Total results: {total_results}")

else:
    print('Failed to fetch data', response.status_code)



# loop through all the data and add to it the 'all_results' open list 
for x in range(0,total_results,limit):
    if x >= total_results:
        break  # this is to prevent the issue of skipping past the results


    site_map = requests.get(f"https://api.fda.gov/drug/ndc.json?api_key={api_key}&limit={limit}&skip={x}")

    if site_map.status_code == 200:
        data = site_map.json()
        if 'results' in data:
            all_results.extend(data['results'])

    else:
        print(f'failed to get data starting at {x} with a status code of {site_map.status_code}')
        print(site_map.text)
        break

    # short delay to avoid rate limit
    time.sleep(1) 

# print out the count
print(f"Final total results: {len(all_results)}")

Total results: 129040
failed to get data starting at 26000 with a status code of 400
{
  "error": {
    "code": "BAD_REQUEST",
    "message": "Skip value must 25000 or less."
  }
}
Final total results: 26000


In [3]:
import psycopg2
from dotenv import load_dotenv
import os

# load credentials
load_dotenv()
db_user = os.getenv('db_user')
db_password = os.getenv('db_password')

# connect to the postgres database
conn = psycopg2.connect(
    host = 'localhost',
    database = 'postgres',
    user = db_user,
    password = db_password)

cursor = conn.cursor()

In [4]:
# reconnect to the access the new database
conn = psycopg2.connect(
    host = 'localhost',
    database = 'Postgres 16 - Localhost - FDA-NDC-ETL_db',
    user = db_user,
    password = db_password)

cursor = conn.cursor()

In [5]:
# create the table
cursor.execute(
'''
CREATE TABLE IF NOT EXISTS FDA_Drugs_db (
    product_ndc VARCHAR,
    generic_name TEXT,
    labeler_name TEXT,
    brand_name TEXT,
    active_ingredients JSON,  -- This needs to be serialized
    finished BOOLEAN,
    packaging JSON,  -- This needs to be serialized. Breakup formatting and make FK
    listing_expiration_date DATE,
    openfda JSON, -- This needs to be serialized
    marketing_category VARCHAR,
    dosage_form VARCHAR,
    spl_id VARCHAR,
    product_type VARCHAR,
    route TEXT[],
    marketing_start_date DATE,
    product_id VARCHAR PRIMARY KEY,
    application_number VARCHAR,
    brand_name_base VARCHAR,
    pharm_class TEXT[],  -- This needs to be serialized
    etl_run_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                                        );
'''
)
# Commit
conn.commit()

In [6]:
# Using a manual approach for data extraction and handling to break out each item for SQL insertion instead of relying on pd.json_normalize
for result in all_results:
    product_ndc = result.get('product_ndc')
    generic_name = result.get('generic_name')
    labeler_name = result.get('labeler_name')
    brand_name = result.get('brand_name')
    active_ingredients = json.dumps(result.get('active_ingredients'))  # this is a dictionary
    finished = result.get('finished')
    packaging = json.dumps(result.get('packaging'))  # this is a dictionary
    listing_expiration_date = result.get('listing_expiration_date')
    openfda = json.dumps(result.get('openfda')) # this is a dictionary
    marketing_category = result.get('marketing_category')
    dosage_form = result.get('dosage_form')
    spl_id = result.get('spl_id')
    product_type = result.get('product_type')
    route = result.get('route')
    marketing_start_date = result.get('marketing_start_date')
    product_id = result.get('product_id')
    application_number = result.get('application_number')
    brand_name_base = result.get('brand_name_base')
    pharm_class = result.get('pharm_class') # this is a dictionary


    # Add data to the table
    cursor.execute(
        '''
        INSERT INTO FDA_Drugs_db (
        "product_ndc",
        "generic_name",
        "labeler_name",
        "brand_name",
        "active_ingredients",
        "finished",
        "packaging",
        "listing_expiration_date",
        "openfda",
        "marketing_category",
        "dosage_form",
        "spl_id",
        "product_type",
        "route",
        "marketing_start_date",
        "product_id",
        "application_number",
        "brand_name_base",
        "pharm_class"
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT ("product_id") DO NOTHING
        ''', 
        (
            product_ndc,
            generic_name,
            labeler_name,
            brand_name,
            active_ingredients,
            finished,
            packaging,
            listing_expiration_date,
            openfda,
            marketing_category,
            dosage_form,
            spl_id,
            product_type,
            route,
            marketing_start_date,
            product_id,
            application_number,
            brand_name_base,
            pharm_class
        )
    )

    # Commit the transaction
    conn.commit()
print("Data inserted successfully")

Data inserted successfully


In [7]:
# add data to the table
sql_query = '''
            SELECT * FROM FDA_Drugs_db
            '''
    
# Execute the query
cursor.execute(sql_query)
# Fetch all the results
results = cursor.fetchall()

In [8]:
import pandas as pd

# Add column names to dataframe
column_names = [x[0] for x in cursor.description]
df = pd.DataFrame(results, columns=column_names)

# See results
df.head()

,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,marketing_category,dosage_form,spl_id,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,pharm_class,etl_run_date
0,79481-5030,"Acetaminophen, Dextromethorphan HBr, Guaifenes...","Meijer Distribution, Inc.",Cold and Flu Severe,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '79481-5030-8', 'description'...",2025-12-31,"{'manufacturer_name': ['Meijer Distribution, I...",OTC MONOGRAPH DRUG,"TABLET, FILM COATED",d8418012-42d2-4d25-a7a9-e26825787288,HUMAN OTC DRUG,[ORAL],2024-08-21,79481-5030_d8418012-42d2-4d25-a7a9-e26825787288,M012,Cold and Flu,"[Adrenergic alpha1-Agonists [MoA], Decreased R...",2025-02-11 23:09:15.554031
1,79903-105,"Menthol 11%, Camphor 11% Ointment","WALMART, INC",Equate Pain Relief Balm,"[{'name': 'CAMPHOR (SYNTHETIC)', 'strength': '...",True,"[{'package_ndc': '79903-105-18', 'description'...",2025-12-31,"{'manufacturer_name': ['WALMART, INC'], 'rxcui...",OTC MONOGRAPH DRUG,OINTMENT,13bdca7a-a10b-ee06-e063-6394a90a1ecd,HUMAN OTC DRUG,[TOPICAL],2022-04-01,79903-105_13bdca7a-a10b-ee06-e063-6394a90a1ecd,M017,Equate Pain Relief Balm,None,2025-02-11 23:09:15.555721
2,79974-207,"Methyl Salicylate, Menthol","Miramar Cosmetics, Inc DBA Miramar Lab",El Unguento Cebo de Coyote,"[{'name': 'MENTHOL, UNSPECIFIED FORM', 'streng...",True,"[{'package_ndc': '79974-207-05', 'description'...",None,"{'manufacturer_name': ['Miramar Cosmetics, Inc...",OTC MONOGRAPH DRUG,OINTMENT,c4f6fe16-6565-4a39-8124-fa1698c91296,HUMAN OTC DRUG,[TOPICAL],2023-11-15,79974-207_c4f6fe16-6565-4a39-8124-fa1698c91296,M017,El Unguento Cebo de Coyote,None,2025-02-11 23:09:15.556406
3,80425-0103,Clindamycin HCL,"Advanced Rx of Tennessee, LLC",Clindamycin HCL,"[{'name': 'CLINDAMYCIN HYDROCHLORIDE', 'streng...",True,"[{'package_ndc': '80425-0103-1', 'description'...",2025-12-31,{'manufacturer_name': ['Advanced Rx of Tenness...,ANDA,CAPSULE,2a99aae5-c901-7940-e063-6294a90a4146,HUMAN PRESCRIPTION DRUG,[ORAL],2001-03-23,80425-0103_2a99aae5-c901-7940-e063-6294a90a4146,ANDA065061,Clindamycin HCL,"[Decreased Sebaceous Gland Activity [PE], Linc...",2025-02-11 23:09:15.557103
4,80425-0350,"Methylprednisolone Acetate, Lidocaine Hydrochl...","Advanced Rx Pharmacy of Tennessee, LLC",Dyural 80-LM,None,True,"[{'package_ndc': '80425-0350-1', 'description'...",2025-12-31,{'manufacturer_name': ['Advanced Rx Pharmacy o...,UNAPPROVED DRUG OTHER,KIT,2a9bfe6a-6d76-b262-e063-6294a90a1cf3,HUMAN PRESCRIPTION DRUG,"[EPIDURAL, INFILTRATION, INTRA-ARTICULAR, INTR...",2023-07-31,80425-0350_2a9bfe6a-6d76-b262-e063-6294a90a1cf3,None,Dyural 80-LM,None,2025-02-11 23:09:15.557853
